In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px
import pandas as pd
from datetime import timedelta


In [90]:
transacciones = pd.read_csv("/Users/carlosoliver/Desktop/EDA_xnova_internacional/Entrega_EDA_Carlos_Oliver/US_SAMPLE.csv", sep= ",")

In [91]:
columns_to_select = ['consignee_address', 'product_desc', 'carrier_sasc_code', 'inbond_type',
                     'loading_port', 'unloading_port', 'container_type']

filtro = transacciones[columns_to_select].iloc[0:1,:]
filtro

,consignee_address,product_desc,carrier_sasc_code,inbond_type,loading_port,unloading_port,container_type
0,314 W SUPERIOR ST SUITE 201,CERAMIC TILES,"GNSI, GOUGH & SONS INC",0.0,"47537, LIVORNO","1703, SAVANNAH, GA",2210


In [92]:
transacciones.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129165 entries, 0 to 1129164
Data columns (total 35 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   complete_date          1129165 non-null  object 
 1   bill_of_lading         1129165 non-null  object 
 2   country                1128968 non-null  object 
 3   country_sure_level     1129165 non-null  int64  
 4   weight                 1129039 non-null  float64
 5   shipper_name           1129165 non-null  object 
 6   shipper_address        1129163 non-null  object 
 7   consignee_name         1129165 non-null  object 
 8   consignee_address      1129165 non-null  object 
 9   product_desc           1129154 non-null  object 
 10  hs_code                1129165 non-null  int64  
 11  hs_code_sure_level     1129165 non-null  int64  
 12  estimate_arrival_date  1129165 non-null  int64  
 13  carrier_sasc_code      1129165 non-null  object 
 14  vessel_country_cod

In [93]:
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import joblib
import xgboost as xgb
from datetime import timedelta

# Suprimir los warnings
warnings.filterwarnings("ignore")


In [94]:

# Convertir columnas de fecha y calcular el tiempo de retraso
transacciones["estimate_arrival_date"] = pd.to_datetime(transacciones["estimate_arrival_date"].astype(str), format='%Y%m%d')
transacciones["complete_date"] = pd.to_datetime(transacciones["complete_date"])
transacciones['delay_time'] = transacciones['complete_date'] - transacciones['estimate_arrival_date']

# Define el rango razonable para el tiempo de retraso (en días)
min_delay = timedelta(days=1)
max_delay = timedelta(days=30)

# Filtra los datos para eliminar valores atípicos
df = transacciones[(transacciones['delay_time'] >= min_delay) & (transacciones['delay_time'] <= max_delay)]

# Eliminar columnas innecesarias
df_clean = df.drop(['bill_of_lading', 'country_sure_level', 'marks_numbers', 'true_supplier', 'true_buyer', 'manifest_n', 'day', 'states', 
                    'year', 'hs_code_sure_level', 'hs_code', 'container_id', 'quantity_unit', 'notify_party_name', 'voyage', 
                    'vessel_country_code', 'vessel_code', 'quantity', 'place_of_receipt', 'vessel_name', 'consignee_name', 
                    'shipper_address', 'shipper_name', 'weight', 'country', 'month', 'product_desc', 'inbond_type', 'container_type'], axis=1)
pd.options.display.max_columns = None

df_clean = df_clean.head(2000)


In [95]:
df_clean

,complete_date,consignee_address,estimate_arrival_date,carrier_sasc_code,loading_port,unloading_port,delay_time
0,2019-03-02,314 W SUPERIOR ST SUITE 201,2019-02-24,"GNSI, GOUGH & SONS INC","47537, LIVORNO","1703, SAVANNAH, GA",6 days
2,2019-12-14,200 WEST SAND LAKE ROAD SUITE 800,2019-12-13,OPLT,"58309, KAO HSIUNG","5201, MIAMI, FL",1 days
4,2019-12-16,7555 BEEDIE WAY DELTA BC V4G0A5 CA,2019-12-15,"SHKK, SCHOOKS TRUCKING","57035, SHANGHAI","3001, SEATTLE, WA",1 days
6,2019-10-24,191 PARKWAY WEST DUNCAN SC 29334 US,2019-10-23,"BANQ, KUEHNE & NAGEL INC.","42870, BREMERHAVEN","1601, CHARLESTON, SC",1 days
7,2019-12-01,C O L1M SUPPLY INC 1648 NW 34TH TERRACE,2019-11-29,OPLT,"58309, KAO HSIUNG","5201, MIAMI, FL",2 days
...,...,...,...,...,...,...,...
3145,2020-01-07,530 7TH AVE NEW YORK N Y 10018 USA,2020-01-06,"APLU, AMERICAN PRESIDENT LINES LTD","57035, SHANGHAI","2704, LOS ANGELES, CA",1 days
3147,2020-01-04,425 106TH AVE NE BELLEVUE WA 98004 USA BELLEVU...,2019-12-27,APVF,"58309, KAO HSIUNG","3001, SEATTLE, WA",8 days
3148,2020-01-18,6341 TUCKAWAY COVE AVE LAS VEGAS LAS VEGAS 891...,2020-01-16,APXL,"55206, VUNG TAU","2709, LONG BEACH, CA",2 days
3149,2020-01-19,0,2020-01-16,"ASFN, ATLANTA SPECIALTY FOOD DISTRIBUTORS","55206, VUNG TAU","1703, SAVANNAH, GA",3 days


In [96]:

# Función para categorizar el retraso
def categorize_delay(days):
    if days <= timedelta(days=10):
        return 0
    elif days <= timedelta(days=20):
        return 1
    else:
        return 2


In [97]:

# Aplicar la función de categorización y eliminar columnas innecesarias
df_clean['delay_category'] = df_clean['delay_time'].apply(categorize_delay)
df_clean.drop(['delay_time', 'complete_date'], axis=1, inplace=True)

# Convertir columnas de fecha en características numéricas
df_clean['estimate_arrival_date_year'] = df_clean['estimate_arrival_date'].dt.year
df_clean['estimate_arrival_date_month'] = df_clean['estimate_arrival_date'].dt.month
df_clean['estimate_arrival_date_day'] = df_clean['estimate_arrival_date'].dt.day
df_clean.drop(['estimate_arrival_date'], axis=1, inplace=True)

# Separar las características de la variable objetivo
X = df_clean.drop(['delay_category'], axis=1)
y = df_clean['delay_category']


In [123]:
X

,consignee_address,carrier_sasc_code,loading_port,unloading_port,estimate_arrival_date_year,estimate_arrival_date_month,estimate_arrival_date_day
0,314 W SUPERIOR ST SUITE 201,"GNSI, GOUGH & SONS INC","47537, LIVORNO","1703, SAVANNAH, GA",2019,2,24
2,200 WEST SAND LAKE ROAD SUITE 800,OPLT,"58309, KAO HSIUNG","5201, MIAMI, FL",2019,12,13
4,7555 BEEDIE WAY DELTA BC V4G0A5 CA,"SHKK, SCHOOKS TRUCKING","57035, SHANGHAI","3001, SEATTLE, WA",2019,12,15
6,191 PARKWAY WEST DUNCAN SC 29334 US,"BANQ, KUEHNE & NAGEL INC.","42870, BREMERHAVEN","1601, CHARLESTON, SC",2019,10,23
7,C O L1M SUPPLY INC 1648 NW 34TH TERRACE,OPLT,"58309, KAO HSIUNG","5201, MIAMI, FL",2019,11,29
...,...,...,...,...,...,...,...
3145,530 7TH AVE NEW YORK N Y 10018 USA,"APLU, AMERICAN PRESIDENT LINES LTD","57035, SHANGHAI","2704, LOS ANGELES, CA",2020,1,6
3147,425 106TH AVE NE BELLEVUE WA 98004 USA BELLEVU...,APVF,"58309, KAO HSIUNG","3001, SEATTLE, WA",2019,12,27
3148,6341 TUCKAWAY COVE AVE LAS VEGAS LAS VEGAS 891...,APXL,"55206, VUNG TAU","2709, LONG BEACH, CA",2020,1,16
3149,0,"ASFN, ATLANTA SPECIALTY FOOD DISTRIBUTORS","55206, VUNG TAU","1703, SAVANNAH, GA",2020,1,16


In [101]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Define las columnas categóricas y numéricas en el DataFrame X
categorical_columns = X.select_dtypes(include=['object']).columns
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Definir la transformación para características numéricas
# - Imputar valores faltantes con la media
# - Escalar los valores numéricos al rango [0, 1]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Definir el transformador para características categóricas
# - Aplicar OneHotEncoding para convertir categorías en vectores binarios
# - Manejar categorías desconocidas durante la predicción
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Definir el preprocesador que aplica transformaciones específicas a las columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),  # Aplicar transformaciones numéricas
        ('cat', categorical_transformer, categorical_columns)  # Aplicar transformaciones categóricas
    ]
)

# Definir el modelo de clasificación
# - Configurar el clasificador XGBoost con parámetros básicos
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Crear el pipeline que integra el preprocesador y el modelo
# - El preprocesador transforma los datos
# - El modelo se ajusta a los datos transformados
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# El pipeline está ahora listo para ser entrenado y evaluado


In [125]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar el pipeline a los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Guardar el pipeline entrenado en un archivo pickle
joblib.dump(pipeline, 'pipeline_model.pkl')


Accuracy: 0.9425


['pipeline_model.pkl']

In [127]:
# Importar librerías necesarias
import pandas as pd
import joblib
import numpy as np

# Datos de entrada para la predicción
X_new = pd.DataFrame({
    'consignee_address': ['314 W SUPERIOR ST SUITE 201'],
    'carrier_sasc_code': ['GNSI, GOUGH & SONS INC'],
    'loading_port': ['47537, LIVORNO'],
    'unloading_port': ['1703, SAVANNAH, GA'],
    'estimate_arrival_date_year': [2024],
    'estimate_arrival_date_month': [10],
    'estimate_arrival_date_day': [20]
})

# Cargar el pipeline desde el archivo pickle
pipeline = joblib.load('pipeline_model.pkl')

# Realizar la predicción
y_pred_new = pipeline.predict(X_new)

# Mostrar la predicción
print(f"La predicción para los datos nuevos es: {y_pred_new[0]} se podrá retrasar hasta 10 días pero entra en el tiempo")


La predicción para los datos nuevos es: 0 se podrá retrasar hasta 10 días pero entra en el tiempo


In [1]:
# Abrir el archivo en modo lectura ('r')
archivo = open('AmazonCost-b342c55c-8d3c-43da-95c5-8cb87b8e589c (1) (1).csv', 'r')


FileNotFoundError: [Errno 2] No such file or directory: 'AmazonCost-b342c55c-8d3c-43da-95c5-8cb87b8e589c (1) (1).csv'

In [ ]:

# Leer el contenido del archivo
contenido = archivo.read()

# Mostrar el contenido
print(contenido)

# Cerrar el archivo
archivo.close()